In [1]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from E:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


# 创建 Procedure

用户自定义的函数有个缺点：不能在事务中使用。因为函数的执行会自动提交事务。为了解决这个问题，PostgreSQL 提供了 Procedure。`stored procedure` 能够解决这个问题。Procedure 是一种特殊的函数，它能够在事务中使用。

语法：
```
create [or replace] procedure procedure_name(parameter_list)
language plpgsql
as $$
declare
-- variable declaration
begin
-- stored procedure body
end; $$
```

In [2]:
sql = """
drop table if exists accounts;

create table accounts (
    id int generated by default as identity,
    name varchar(100) not null,
    balance dec(15,2) not null,
    primary key(id)
);

insert into accounts(name,balance)
values('Bob',10000);

insert into accounts(name,balance)
values('Alice',10000);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1d20a9716f0>

In [3]:
sql = """
select * from accounts;
"""
run_sql(cursor, sql)

   id   name   balance
0   1    Bob  10000.00
1   2  Alice  10000.00


In [4]:
sql = """
create or replace procedure transfer(
   sender int,
   receiver int, 
   amount dec
)
language plpgsql    
as $$
begin
    -- subtracting the amount from the sender's account 
    update accounts 
    set balance = balance - amount 
    where id = sender;

    -- adding the amount to the receiver's account
    update accounts 
    set balance = balance + amount 
    where id = receiver;
end;$$;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1d20a9716f0>

In [5]:
sql = """
call transfer(1,2,1000);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1d20a9716f0>

In [6]:
sql = """
SELECT * FROM accounts;
"""
run_sql(cursor, sql)

   id   name   balance
0   1    Bob   9000.00
1   2  Alice  11000.00
